In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.getOrCreate()

24/05/20 22:31:28 WARN Utils: Your hostname, Alan-Youngs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.51 instead (on interface en0)
24/05/20 22:31:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/20 22:31:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql import SQLContext

In [4]:
sqlcontext = SQLContext(spark)

/Users/alanyoung/Documents/code/code/pythoncode/venv/lib/python3.11/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
snapshot_file = 'data/2024-05-15-snapshot1_transactions.csv'

In [6]:
initial_file = 'data/initialList.csv'

In [7]:

snapshot_df = sqlcontext.read.load(snapshot_file,
                      format='com.databricks.spark.csv',
                      header='true',
                      inferSchema='true')

24/05/20 22:31:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [8]:
snapshot_df.registerTempTable("snapshot")


/Users/alanyoung/Documents/code/code/pythoncode/venv/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+--------------------+-----------------------+--------------------+-----------------+----------------------------+--------------------+--------------------+--------------------+--------------+---------------+-----------------+
|        SOURCE_CHAIN|SOURCE_TRANSACTION_HASH|     SOURCE_CONTRACT|DESTINATION_CHAIN|DESTINATION_TRANSACTION_HASH|DESTINATION_CONTRACT|       SENDER_WALLET|SOURCE_TIMESTAMP_UTC|       PROJECT|NATIVE_DROP_USD|STARGATE_SWAP_USD|
+--------------------+-----------------------+--------------------+-----------------+----------------------------+--------------------+--------------------+--------------------+--------------+---------------+-----------------+
|Klaytn Mainnet Cy...|   0x35cbf0b8b70c8be...|0x79db0f1a83f8e74...|              DFK|        0x3a910f9984792c6...|0x457fd60ffa26576...|0xda235bcd17f416f...| 2023-12-31 06:47:21|        Merkly| 6.102727778E-6|             NULL|
|            Arbitrum|   0x222911eb3865d12...|0x4ae8cebccd70278...|    Meter Mainnet|       

In [9]:
# test_df = sqlcontext.sql("""SELECT *
#                             from snapshot
#                                 WHERE 
#                                 PROJECT NOT IN ('Merkly') AND NATIVE_DROP_USD >0
#                                 """)
# test_df.show()

In [10]:
# initial list
filter_df = sqlcontext.read.load(initial_file,
                      format='com.databricks.spark.csv',
                      header='true',
                      inferSchema='true')
filter_df.registerTempTable("filter")

In [11]:
#1. one wallet sending tokens/ NFT to and/ or from 4 other wallet addresses or more (prior to Snapshot 
#2. Wallet address only has interactions with Layerzero (L0) without other non-L0's on-chain activities 
#3. Wallets only have interactions with Merkly of ONFT bridge or OFT bridge (gas refuel smart contract is not included)
#4. (OR) Wallets with less than $100 worth of balances (as at Snapshot )
#5. Wallets with all L0 transactions with less than $1 as at Snapshot#1 date & time (prior to Snapshot#1)


#a. Only use Merkly, NO refuel; - 

      # date_diff('day', min(block_time), now()) as lz_age_days,
      #   count(distinct source_chain_id) as active_source_chain_count,
      #   count(distinct destination_chain_id) as active_destination_chain_count,
      #   count(distinct transaction_contract) as active_transaction_contract_count,
      #   count(distinct date_trunc('day', block_time)) as active_days_count,
      #   count(distinct date_trunc('week', block_time)) as active_weeks_count,
      #   count(distinct date_trunc('month', block_time)) as active_months_count,
      #   -- coalesce(sum(amount_usd / power(10, p.decimals) * p.price), 0) as amount_usd
      #   coalesce(sum(amount_usd), 0) as amount_usd

                        # SELECT SENDER_WALLET, 
                        #         count(1) as ntx,
                        #         count(distinct SOURCE_CHAIN) as active_source_chain_count, 
                        #         count(distinct DESTINATION_CHAIN) as active_destination_chain_count,
                        #         coalesce(SUM(NATIVE_DROP_USD),0) as amt,
                        #         SUM(NATIVE_DROP_USD) / count_if(NATIVE_DROP_USD is not NULL) as amt_per_tx,
                        #         min(SOURCE_TIMESTAMP_UTC) as ibt,
                        #         max(SOURCE_TIMESTAMP_UTC) as lbt
                        # coalesce(SUM(NATIVE_DROP_USD) / count_if(NATIVE_DROP_USD is not NULL),0) as amt_per_tx,
                        # coalesce(SUM(STARGATE_SWAP_USD) / count_if(STARGATE_SWAP_USD is not NULL),0) as samt_per_tx,
                        #     from snapshot
                        #         WHERE project = 'Merkly'
                        #         GROUP BY SENDER_WALLET 

summary_df = sqlcontext.sql("""SELECT SENDER_WALLET, 
                                count(*) as ntx,  
                                coalesce(SUM(NATIVE_DROP_USD),0) as amt,
                                coalesce(SUM(STARGATE_SWAP_USD),0) as samt,
                                min(SOURCE_TIMESTAMP_UTC) as ibt,
                                max(SOURCE_TIMESTAMP_UTC) as lbt
                            from snapshot
                                WHERE 
                                Group by SENDER_WALLET, project
                                """)
summary_df.registerTempTable("summary")

summary_df.show()


24/05/20 22:31:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:31:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:31:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:31:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:31:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:31:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:31:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:31:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:31:53 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+---+--------------------+------------------+-------------------+-------------------+-----------------+
|       SENDER_WALLET|ntx|                 amt|              samt|                ibt|                lbt|          project|
+--------------------+---+--------------------+------------------+-------------------+-------------------+-----------------+
|0xb01c8c1c03e7338...|  8|                 0.0|               0.0|2023-06-17 04:19:18|2023-08-18 18:44:43|   Harmony Bridge|
|0x8502e510649afae...| 51|                 0.0|13180.911394904997|2023-06-30 09:32:40|2024-04-23 17:18:18|         Stargate|
|0xd49ccb7e24f08c9...| 68|                 0.0|               0.0|2023-05-28 12:01:41|2023-08-26 03:58:30|          CoreDAO|
|0x458ceac0292a63c...| 26|        6.3383681877| 74735.04801085498|2023-04-11 14:45:01|2024-04-30 07:58:26|         Stargate|
|0x27ce77e06d39320...| 34|                 0.0|               0.0|2023-05-27 13:29:27|2023-12-22 19:28:32|   Harmony Bridge|


+--------------------+---+--------------------+--------------------+--------------------+-------------------+-------------------+
|       SENDER_WALLET|ntx|   DESTINATION_CHAIN|                 amt|          amt_per_tx|                ibt|                lbt|
+--------------------+---+--------------------+--------------------+--------------------+-------------------+-------------------+
|0xca1bbd005748112...|  2|                Zora|       0.52510908322|       0.26255454161|2023-12-31 08:18:17|2024-01-14 19:37:07|
|0x8d1bbece2483d7c...|  5|                Kava|     2.0232485325E-5|      4.046497065E-6|2023-07-21 17:04:51|2023-07-23 11:04:05|
|0x64c5371deaa790b...|  4|                 DFK|                 0.0|                 0.0|2023-12-29 23:43:15|2024-04-29 08:36:04|
|0x3022752a17eb5ae...| 17|      Conflux eSpace|   1.22940497359E-12|7.231793962294118...|2023-10-24 11:42:01|2023-10-27 00:11:32|
|0xa375e2077817d01...|  2|Klaytn Mainnet Cy...|        0.0024059613|       0.00120298065|2

In [29]:
#merkly_contract = ['0x6e55472109e6abe4054a8e8b8d9edffcb31032c5', '0xb5691e49f86CBa649c815Ee633679944b044BC43', 0xC7CC66a88e2f121Fb104344EAcB7BA7bCAe79DFA, 0x97337A9710BEB17b8D77cA9175dEFBA5e9AFE62e, 0xFDc9018aF0E37AbF89233554C937eB5068127080, 0x885ef5813E46ab6EFb10567b50b77aAAD4d258ce, 0x6BEd67cd99a3BdB9db561173F5B998230085fe5d, 0x5f45Cd59BA7F2f6bcD935663F68Ee1dEbE3B8a10, 0x5200543580e7ad49FBCb4690c4556D3a6A022584, 0xE92E1BeB21Af5CB887EB27171f18dA14616f102e, 0x5f45Cd59BA7F2f6bcD935663F68Ee1dEbE3B8a10, 0x5200543580e7ad49FBCb4690c4556D3a6A022584, 0x5f45cd59ba7f2f6bcd935663f68ee1debe3b8a10, 0x5f45cd59ba7f2f6bcd935663f68ee1debe3b8a10, 0xDB3Bb6D5a8EeEAfc64C66C176900E6B82b23dd5f, 0x1193Fa4CA41c36CB20dd28c607AeE2c66133d783, 0xb004C6399bBD9Cf6f21Ae98eFd932e6c12f4870a, 0x5200543580e7ad49FBCb4690c4556D3a6A022584, 0x5200543580e7ad49fbcb4690c4556d3a6a022584, 0x1c2150c8639d0Ef4dff3642B99D686cc339e51b8, 0x6BEd67cd99a3BdB9db561173F5B998230085fe5d, 0x0BF0d6260fb4f4FC77b143eaFF9E4Fe51dD08C83, 0x5f45Cd59BA7F2f6bcD935663F68Ee1dEbE3B8a10, 0xCA230856343C300f0cc2Bd77C89F0fCBeDc45B0f, 0x5f45Cd59BA7F2f6bcD935663F68Ee1dEbE3B8a10, 0x016ab078bD8D452Bfbb6740D2b5eE15ae0009a08, 0x5200543580e7ad49FBCb4690c4556D3a6A022584, 0xDB3Bb6D5a8EeEAfc64C66C176900E6B82b23dd5f, 0x5200543580e7ad49FBCb4690c4556D3a6A022584, 0xE47b05F2026a82048caAECf5caE58e5AAE2405eA, 0x766b7aC73b0B33fc282BdE1929db023da1fe6458, 0x5200543580e7ad49FBCb4690c4556D3a6A022584, 0x5200543580e7ad49FBCb4690c4556D3a6A022584, 0x6647D82ec9749Fa697f360a0d9797E449c8E01c4, 0x5200543580e7ad49FBCb4690c4556D3a6A022584, 0x6Ca118bdF9BD900Da5D3d85094D92C5B3b9c0DA5, 0x6Ca118bdF9BD900Da5D3d85094D92C5B3b9c0DA5, 0x5200543580e7ad49FBCb4690c4556D3a6A022584, 0xAFa5f9313F1F2b599173f24807a882F498Be118c, 0xAFa5f9313F1F2b599173f24807a882F498Be118c, 0x6f6aE8851a460406bBB3c929a415d2Df9305AcD5, 0xB89fA5d6A97f5491e7D1D5F418d5b5c64f081D32, 0x6Ca118bdF9BD900Da5D3d85094D92C5B3b9c0DA5, 0x20A66B2Ba6d2DaD5207B4928F7c2142FBC8bAFe8, 0x43Ec7871f9bBba864b4bFBeB404004EBDCe3c259, 0x6f6aE8851a460406bBB3c929a415d2Df9305AcD5, 0xFDc9018aF0E37AbF89233554C937eB5068127080, 0xE030543b943bdCd6559711Ec8d344389C66e1D56, 0xa184998eC58dc1dA77a1F9f1e361541257A50CF4, 0xAa58e77238f0E4A565343a89A79b4aDDD744d649, 0xa2C203d7EF78ed80810da8404090f926d67Cd892, 0x97337A9710BEB17b8D77cA9175dEFBA5e9AFE62e, 0xf1A3BCb8c74aA8ef255E0a358d98a9E2b4A97A59, 0x885ef5813E46ab6EFb10567b50b77aAAD4d258ce, 0xE33519C400B8F040E73aeDa2f45DfDD4634A7cA0, 0x766b7aC73b0B33fc282BdE1929db023da1fe6458, 0xFFd57B46BD670B0461c7C3EBBaEDC4CdB7c4FB80, 0xb58f5110855fbef7a715d325d60543e7d4c18143, 0xd02ffae68d902453b44a9e45dc257aca54fb88b2, 0x2E228120c0AF2dE3A74D744B25B24D1fb28CE5B4, 0xCA230856343C300f0cc2Bd77C89F0fCBeDc45B0f, 0x885ef5813E46ab6EFb10567b50b77aAAD4d258ce, 0xa0a54dADc2a1F198C58Fd0739BA7dF40Ffd366Dc, 0xb58f5110855fBEF7A715d325D60543E7D4c18143, 0x426A8Dc7263A439e92972eE2200DA21EC6cEEcfa, 0xf836905A5C6E9dFBB88c4a9ADf79f6a47B43eCBB, 0x6dd28C2c5B91DD63b4d4E78EcAC7139878371768, 0x97337A9710BEB17b8D77cA9175dEFBA5e9AFE62e, 0x83d8476eBccf8094d80D7b2165375a3Ec4E93034, 0x484c402B0c8254BD555B68827239BAcE7F491023, 0xd81a2e87232b4fdd27fbe16107d8deaaa2d14181, 0x04866796aabB6B58e6bC4d91A2aE99105b2C58AE, 0x5200543580e7ad49FBCb4690c4556D3a6A022584, 0xDB3Bb6D5a8EeEAfc64C66C176900E6B82b23dd5f, 0xF882c982a95F4D3e8187eFE12713835406d11840, 0xFFd57B46BD670B0461c7C3EBBaEDC4CdB7c4FB80, 0xE47b05F2026a82048caAECf5caE58e5AAE2405eA, 0x5f45Cd59BA7F2f6bcD935663F68Ee1dEbE3B8a10, 0x85b1C838E38203E420d59ad84d2f53E14b2e50C3, 0xE47b05F2026a82048caAECf5caE58e5AAE2405eA, 0xc072c3EbAf165955C5aAd2DbB4293f771de6dbd3, 0xc62C04F7CD4B47729027c138FABD99aFA5db1222, 0xDB3Bb6D5a8EeEAfc64C66C176900E6B82b23dd5f, 0x7dFb5E7808B5eb4fB8b9e7169537575f6fF1a218, 0x6E55472109E6aBE4054a8E8b8d9EdFfCb31032C5, 0x6E55472109E6aBE4054a8E8b8d9EdFfCb31032C5, 0xC29086BE89557469ae915dd9Dcaa6c093ab4648A, 0xc072c3EbAf165955C5aAd2DbB4293f771de6dbd3, 0x6E2d829db8EF2cf075903AD062E06066aaFa760F, 0x1712147B84f91A389d2f843B8b3Cc675E6648d80, 0xc072c3ebaf165955c5aad2dbb4293f771de6dbd3, 0xe341f30Ea040bF3691aA069B8c5c213F72676421, 0x70ea00aB512d13dAc5001C968F8D2263d179e2D2, 0x70ea00aB512d13dAc5001C968F8D2263d179e2D2, 0x8a555e4Fc287650f5E8CA1778A35dd44e893d6Aa, 0x20279b6d57Ba6D3eF852f34800e43e39d46d6487, 0xA07f2e99eAa338aCF66337Baf99551BDCFd3AB00, 0xDB3Bb6D5a8EeEAfc64C66C176900E6B82b23dd5f, 0x909499E536DC85cA16b211053bba805C6a46748C, 0x555a37E52791904c097ECAB03da0009a0c7c4FCf, 0x9A44124b18Bcae851134444eE35c53daa77f86c6, 0xE33519C400B8F040E73aeDa2f45DfDD4634A7cA0, 0x54b3297e1C739fe62be5c00473A2e6bb78adB0Ef, 0x6d8eEEeB90F8bA283AA8BEf90C2f78e4aA191BF4, 0x5D425630796346F7d054C97753AE241fc592734b, 0xC7CC66a88e2f121Fb104344EAcB7BA7bCAe79DFA, 0x9F40916d0DFb2F8f5FB63D8f76826d09041f2EAE, 0xb5691e49f86CBa649c815Ee633679944b044BC43, 0x921B486cC33580AF7d8208dF1619383470D5dcbe, 0x0e2cFD13566578eAD8296e8b1813619c4FBc5eDc, 0x885ef5813E46ab6EFb10567b50b77aAAD4d258ce, 0x54De43B6bA21A5553697a2b78338e046dD7E0278, 0x8582A8f68fAf6C2C5B5f4a1eB28122AcF09FeFeE, 0x021B4878Be1cE222E5d1625ED7dbCb7Cd80cF245, 0x148CaF6FfBaBa15F35deE7e2813d1F4c6da288F3, 0x1aaAc4484402C026B30D3D4272e99679664a18FC, 0x921B486cC33580AF7d8208dF1619383470D5dcbe, 0x5f45Cd59BA7F2f6bcD935663F68Ee1dEbE3B8a10, 0x5f45Cd59BA7F2f6bcD935663F68Ee1dEbE3B8a10, 0x5f45Cd59BA7F2f6bcD935663F68Ee1dEbE3B8a10, 0xD838D5b87439e17B0194fd43e37300cD99Aa3DE0, 0x5f45Cd59BA7F2f6bcD935663F68Ee1dEbE3B8a10, 0xDB3Bb6D5a8EeEAfc64C66C176900E6B82b23dd5f, 0xeA1684aeC868CdB4d625F67c02F73D10D7720744, 0xDB3Bb6D5a8EeEAfc64C66C176900E6B82b23dd5f, 0xD31002B35D527c90A45EdeaB6a9Cc288A7997781, 0x4fc0D96f3d70b4D9b75671Ab92e7Be01CaBE3863, 0x5f45Cd59BA7F2f6bcD935663F68Ee1dEbE3B8a10, 0x6E55472109E6aBE4054a8E8b8d9EdFfCb31032C5, 0x1a7206F9c315720d1E5b536B492c448863EB298A, 0xAFa5f9313F1F2b599173f24807a882F498Be118c, 0xb4DBF7eFE78aCf8e49a8ddBC2a9B9fC2836022A8, 0x4fc0D96f3d70b4D9b75671Ab92e7Be01CaBE3863, 0x8c0241C47E42fffD3Cde31eC8ccA32dfD7Ed17F1, 0xB5be9f9db60E39809493DDc47595A0703edA3f2e, 0xafa5f9313f1f2b599173f24807a882f498be118c, 0xeF1eAE0457e8D56A003d781569489Bc5466E574b, 0x5C9BBE51F7F19f8c77DF7a3ADa35aB434aAA86c5, 0x0E1f20075C90Ab31FC2Dd91E536e6990262CF76d, 0x4Ae8CEBcCD7027820ba83188DFD73CCAD0A92806, 0xD7bA4057f43a7C4d4A34634b2A3151a60BF78f0d, 0xF56605276cefffe32DFD8B6bF80B93c2A6840136, 0x457Fd60FFA26576E226252092c98921f12E90FbB, 0x671861008497782F7108D908D4dF18eBf9598b82, 0xC20A842e1Fc2681920C1A190552A2f13C46e7fCF, 0x671861008497782F7108D908D4dF18eBf9598b82, 0xf6b88C4a86965170dd42DBB8b53e790B3490b912, 0x556F119C7433b2232294FB3De267747745A1dAb4, 0x79DB0f1A83f8e743550EeB5DD5B0B83334F2F083, 0xF450A7b8abfc99D1cDEf6656163399aD762d51Cc, 0xa513F61Bc23F0eB1FC0aC4d9dab376d79bC7F3cB, 0x95716171f7737CB8D387c621999589bDD04E6A35, 0x148CaF6FfBaBa15F35deE7e2813d1F4c6da288F3, 0xE62d19Df93D84b3552498260188D19A772296B10, 0x4c24Ba5177365b4c0eBae62b31945d830a858673, 0xb5691e49f86CBa649c815Ee633679944b044BC43, 0x5673B6e6e51dE3479B8deB22dF46B12308db5E1e, 0xd379c3D0930d70022B3C6EBA8217e4B990705540, 0x2935a2804E4F902E722B64C51a831Bce2526a576, 0xB6789dACf323d60F650628dC1da344d502bC41E3, 0xB6c5e0d2ffC3Fc80c8D3F5A8b86b7A796A2c5782, 0x4c24Ba5177365b4c0eBae62b31945d830a858673, 0xE7D454d096d38b22e0C30470e7FB20B1B2aCf70D, 0xc9B753d73B17DDb5E87093ff04A9e31845a43af0, 0x6bf98654205B1AC38645880Ae20fc00B0bB9FFCA, 0x461fcCF240CA4884Cc5413a5742F1bC56fAf7A0C, 0x457Fd60FFA26576E226252092c98921f12E90FbB, 0xE47b05F2026a82048caAECf5caE58e5AAE2405eA, 0x2935a2804E4F902E722B64C51a831Bce2526a576, 0x457Fd60FFA26576E226252092c98921f12E90FbB, 0x4fc0D96f3d70b4D9b75671Ab92e7Be01CaBE3863, 0xf1CD5b12211664FFaf62Af7E5E5C27b1b558aBa2, 0xE47b05F2026a82048caAECf5caE58e5AAE2405eA, 0x6f6aE8851a460406bBB3c929a415d2Df9305AcD5, 0x7dFb5E7808B5eb4fB8b9e7169537575f6fF1a218, 0x7dFb5E7808B5eb4fB8b9e7169537575f6fF1a218, 0xB75Bf6fcdf431a6462a0669326d9118aB48D3307, 0xc62C04F7CD4B47729027c138FABD99aFA5db1222, 0x80cb36d71cFbFbA6602f9692D109aa329D702b6C, 0x5BA99F0E6A42cdfAb564921a382551D8AfEAad61, 0x7dfb5e7808b5eb4fb8b9e7169537575f6ff1a218, 0x7dFb5E7808B5eb4fB8b9e7169537575f6fF1a218, 0x7dfb5e7808b5eb4fb8b9e7169537575f6ff1a218, 0xEF62b433Ca3AC8b151c4a255de3eD3dA4e60AdD2, 0xAFa5f9313F1F2b599173f24807a882F498Be118c, 0x7dFb5E7808B5eb4fB8b9e7169537575f6fF1a218, 0x7dFb5E7808B5eb4fB8b9e7169537575f6fF1a218, 0x7dFb5E7808B5eb4fB8b9e7169537575f6fF1a218, 0x7dFb5E7808B5eb4fB8b9e7169537575f6fF1a218, 0x7dFb5E7808B5eb4fB8b9e7169537575f6fF1a218, 0x7dfb5e7808b5eb4fb8b9e7169537575f6ff1a218, 0x7dFb5E7808B5eb4fB8b9e7169537575f6fF1a218]

In [12]:
# initial list
filter_df = sqlcontext.read.load(initial_file,
                      format='com.databricks.spark.csv',
                      header='true',
                      inferSchema='true')
filter_df.registerTempTable("filter")

In [31]:
#1. one wallet sending tokens/ NFT to and/ or from 4 other wallet addresses or more (prior to Snapshot 
#2. Wallet address only has interactions with Layerzero (L0) without other non-L0's on-chain activities 
#3. Wallets only have interactions with Merkly of ONFT bridge or OFT bridge (gas refuel smart contract is not included)
#4. (OR) Wallets with less than $100 worth of balances (as at Snapshot )
#5. Wallets with all L0 transactions with less than $1 as at Snapshot#1 date & time (prior to Snapshot#1)filter_df.show()

ConnectionRefusedError: [Errno 61] Connection refused

In [13]:
#1. one wallet sending tokens/ NFT to and/ or from 4 other wallet addresses or more (prior to Snapshot 
#2. Wallet address only has interactions with Layerzero (L0) without other non-L0's on-chain activities 
#3. Wallets only have interactions with Merkly of ONFT bridge or OFT bridge (gas refuel smart contract is not included)
#4. (OR) Wallets with less than $100 worth of balances (as at Snapshot )
#5. Wallets with all L0 transactions with less than $1 as at Snapshot#1 date & time (prior to Snapshot#1)


sybil_df = sqlcontext.sql("""SELECT * 
                            from summary
                                WHERE amt < 1
                                AND samt < 1
                                AND SENDER_WALLET NOT IN (SELECT address from filter) """)

In [14]:
import os
sybil_df.write.csv('data/sybil1')
os.system("cat data/sybil1/p* > data/sybil1.csv")
 

24/05/20 22:33:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:33:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:33:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:33:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:33:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:33:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:33:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:33:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/20 22:33:58 WARN RowBasedKeyValueBatch: Calling spill() on

[151.943s][warning][gc,alloc] Executor task launch worker for task 2.0 in stage 10.0 (TID 657): Retried waiting for GCLocker too often allocating 131074 words
[151.943s][warning][gc,alloc] Executor task launch worker for task 1.0 in stage 10.0 (TID 656): Retried waiting for GCLocker too often allocating 131074 words
[151.943s][warning][gc,alloc] Executor task launch worker for task 5.0 in stage 10.0 (TID 660): Retried waiting for GCLocker too often allocating 131074 words
[152.057s][warning][gc,alloc] Executor task launch worker for task 12.0 in stage 10.0 (TID 667): Retried waiting for GCLocker too often allocating 131074 words
[152.062s][warning][gc,alloc] Executor task launch worker for task 14.0 in stage 10.0 (TID 669): Retried waiting for GCLocker too often allocating 131074 words


24/05/20 22:33:59 ERROR Executor: Exception in task 2.0 in stage 10.0 (TID 657)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:71)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:391)
	at org.apache.spark.io.ReadAheadInputStream.<init>(ReadAheadInputStream.java:105)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:74)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.getSortedIterator(UnsafeExternalSorter.java:555)
	at org.apache.spark.sql.execution.UnsafeKVExternalSorter.sortedIterator(UnsafeKVExternalSorter.java:206)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.finishAggregate(HashAggregateExec.scala:223)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.ha

ConnectionRefusedError: [Errno 61] Connection refused

In [46]:
#Cliff Lam: 0x7ba1Fc6ea6aA5D020D440A4576473c71057374c9
#Cliff Lam: 0x0a18cBDc365014c0623BB713bBF42EED9c882DC4
#Cliff Lam: 0x74451a23B838D123B98460Da38ea6bCEE824fF49

# time_df = sqlcontext.sql("""SELECT *
#                             from snapshot
#                                 WHERE 
#                                   SENDER_WALLET = LOWER('0x7ba1Fc6ea6aA5D020D440A4576473c71057374c9')
                                  
#                                 """)

In [52]:
# import os
# time_df.write.csv('data/example')
# os.system("cat data/example/p* > data/example.csv")

0